## PyTorch Linear Regression

Open a terminal and enter:

conda install pytorch torchvision -c pytorch

or 

in code cell, enter:

!conda install pytorch torchvision -c pytorch

PyTorch Documentation

https://pytorch.org/docs/stable/index.html

In [ ]:
import torch
import torch.nn as nn
torch.__version__

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

#### Google colab

Check if using Google Colab in order to upload data files

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

In [ ]:
howell = pd.read_csv("Howell.csv",sep=';')
adult = howell.query("age > 17")
adult.tail()

In [ ]:
X = adult.weight.values.reshape(-1,1)
#X = adult.loc[:,['weight','age']]
y = adult.height.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### Create the linear regression model

The model will have one Linear layer with one input feature and one output prediction.

The model will have a $\_\_call\_\_()$ method which calls the forward function

In [ ]:
torch.manual_seed(1)

# Define the class, the arguments are the number of input features and output preds
class LR(nn.Module):
    def __init__(self,num_in,num_out):
        super(LR, self).__init__() 
        self.linear = torch.nn.Linear(num_in, num_out)  #Instantiate Linear Class
        
    # the functon that is called in the forward pass
    def forward(self, input):   
        y_pred = self.linear(input) 
        return y_pred # return the prediction

# Model Constructor
model = LR(X_train.shape[1],1) #number of features, 1 prediction
model

In [ ]:
type(model.linear)

#### Loss and optimizer

Mean Squared error loss and stochastic gradient descent optimization

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05) #lr = learning rate
optimizer

In [ ]:
model.parameters()

#### Create the tensors for the model

Numpy creates float64 by default, but PyTorch uses float32 by default.

In [ ]:

feats = torch.from_numpy(X_train.astype(np.float32))
target = torch.from_numpy(y_train.astype(np.float32))
test_input = torch.from_numpy(X_test.astype(np.float32))
test_target = torch.from_numpy(y_test.astype(np.float32))

feats.shape,target.shape,test_input.shape,test_target.shape

#### Train the model

Fit the model on the training data and then do a forward pass for the test data

In [ ]:
n_epochs = 30

train_losses = np.zeros(n_epochs)

for i in range(n_epochs):

    optimizer.zero_grad()  # zero the parameter gradients
    outputs = model(feats) # Forward pass
    loss = criterion(outputs, target) # Calculate loss
    loss.backward()        # Backward pass
    optimizer.step()       # Update weights
    train_losses[i] = loss.item() # Log loss
    
    print(f'Epoch {i+1}/{n_epochs}, Train Loss: {loss.item():.4f}')

In [ ]:
y_pred = model(torch.from_numpy(X_test.astype(np.float32))) # Forward pass for test data
type(y_pred)

In [ ]:
predicted2 = y_pred.detach().numpy()
predicted2.shape,type(predicted2),predicted2.dtype

In [ ]:
r2_score(y_test,predicted2),np.sqrt(mean_squared_error(y_test,predicted2))

In [ ]:
if X_test.shape[1] == 1:
    plt.scatter(X_test, y_test, label='Original data')
    plt.plot(X_test, predicted2, label='Fitted2 line')
    plt.legend();

In [ ]:
list(model.parameters())

#### Alternate way to generate predictions for test data

Train and test at the same time

In [ ]:
torch.manual_seed(1)

# Define the class, the arguments are the number of inputs and outputs
class LR(nn.Module):
    def __init__(self,num_in,num_out):
        super(LR, self).__init__() 
        self.linear = torch.nn.Linear(num_in, num_out)  
        
    # the functon that is called in the forward pass
    def forward(self, x):   
        y_pred = self.linear(x) 
        return y_pred # return the prediction

# Model Constructor
model = LR(X_train.shape[1],1) #number of features, 1 prediction


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05) #lr = learning rate

feats = torch.from_numpy(X_train.astype(np.float32))
target = torch.from_numpy(y_train.astype(np.float32))
test_input = torch.from_numpy(X_test.astype(np.float32))
test_target = torch.from_numpy(y_test.astype(np.float32))

feats.shape,target.shape,test_input.shape,test_target.shape

In [ ]:
n_epochs = 30

train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for i in range(n_epochs):

    optimizer.zero_grad()  # zero the parameter gradients
    outputs = model(feats) # Forward pass
    loss = criterion(outputs, target) # Calculate loss
    loss.backward()        # Backward pass
    optimizer.step()       # Update weights
    train_losses[i] = loss.item() # Log loss
    
    outputs_test = model(test_input) # run the test data
    loss_test = criterion(outputs_test, test_target) # Calulate loss
    test_losses[i] = loss_test.item()
    
    print(f'Epoch {i+1}/{n_epochs}, Train Loss: {loss.item():.4f} Test Loss: {loss_test.item():.4f}')

#### Plot the loss per iteration

In [ ]:

plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend();

#### Convert predictions for the test data from tensors to numpy arrays

In [ ]:
predicted = outputs_test.detach().numpy()
predicted.shape

#### Calculate Coefficient of Determination

In [ ]:
r2_score(y_test,predicted),np.sqrt(mean_squared_error(y_test,predicted))

#### Plot the predictions

In [ ]:
if X_test.shape[1] == 1:
    plt.scatter(X_test, y_test, label='Original data')
    plt.plot(X_test, predicted, 'r', label='Fitted line')
    plt.legend();

### Two layers

In [ ]:
X = adult.loc[:,['weight','age']].values
y = adult.height.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=1234)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
torch.manual_seed(1)

class LR2(nn.Module):
    def __init__(self,num_in,num_out):
        super(LR2, self).__init__() 
        self.hid1 = nn.Linear(num_in,8)
        self.outp = nn.Linear(8,num_out)
        
  
    def forward(self, x): 
        z = nn.ReLU()(self.hid1(x))
        y_pred = self.outp(z)
        return y_pred 
    
model = LR2(X_train.shape[1],1)


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
feats = torch.from_numpy(X_train.astype(np.float32))
target = torch.from_numpy(y_train.astype(np.float32))
test_input = torch.from_numpy(X_test.astype(np.float32))
test_target = torch.from_numpy(y_test.astype(np.float32))
feats.shape,target.shape,test_input.shape,test_target.shape

In [ ]:
# Train the model
n_epochs = 30

train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for i in range(n_epochs):

    optimizer.zero_grad()  # zero the parameter gradients
    outputs = model(feats) # Forward pass
    loss = criterion(outputs, target) # Calculate loss
    loss.backward()        # Backward pass
    optimizer.step()       # Update weights
    train_losses[i] = loss.item() # Log loss
    
    outputs_test = model(test_input) # Get test loss
    loss_test = criterion(outputs_test, test_target)
    test_losses[i] = loss_test.item()
    
    print(f'Epoch {i+1}/{n_epochs}, Train Loss: {loss.item():.4f} Test Loss: {loss_test.item():.4f}')

In [ ]:
# Plot the loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend();

In [ ]:
foo = model(torch.from_numpy(X_test.astype(np.float32)))
predicted = foo.detach().numpy()
predicted.shape

In [ ]:
r2_score(y_test,predicted),np.sqrt(mean_squared_error(y_test,predicted))

In [ ]:
list(model.parameters())